In [10]:
import pandas as pd
import ast
import seaborn as sns
from tqdm import tqdm

In [2]:
train = pd.read_csv('/raid/siim-covid19-detection/train_image_level_folded_v2.csv')

In [4]:
train['StudyInstanceUID'].value_counts()

0fd2db233deb    9
a7335b2f9815    7
8943d1d85097    6
970c96c9fa5d    5
a4e94133d95a    5
               ..
f0637f0d7719    1
f0bfb67c18bc    1
6a6071a3d542    1
159c607b0f88    1
51e626885505    1
Name: StudyInstanceUID, Length: 6054, dtype: int64

In [5]:
(train['StudyInstanceUID'].value_counts() > 1).sum()

232

In [20]:
vcs = train['StudyInstanceUID'].value_counts()
study_ids = vcs[vcs > 1].index.values
len(study_ids)

232

In [8]:
train[train['StudyInstanceUID'] == '8943d1d85097']

,id,boxes,label,StudyInstanceUID,series_id,height,width,fold,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
312,0bd6cd815ba9_image,NaN,none 1 0 0 1 1,8943d1d85097,b9820b7c6b80,1760,2140,2,0,1,0,0
1331,35e398a5a431_image,NaN,none 1 0 0 1 1,8943d1d85097,847da36d8859,1760,2140,2,0,1,0,0
2733,6f54e9cbd180_image,NaN,none 1 0 0 1 1,8943d1d85097,4d2a4e3f2904,1760,2140,2,0,1,0,0
4814,c3a09e8a600d_image,NaN,none 1 0 0 1 1,8943d1d85097,24f76a89447b,1760,2140,2,0,1,0,0
4845,c4b68b29a072_image,NaN,none 1 0 0 1 1,8943d1d85097,fdf2e4a2f7d5,1760,2140,2,0,1,0,0
4923,c7925ab50eb0_image,"[{'x': 1265.90376, 'y': 571.61961, 'width': 37...",opacity 1 1265.90376 571.61961 1639.1595 1385....,8943d1d85097,5b9fdac1600e,1760,2140,2,0,1,0,0


In [24]:
bad_images = []
for study_id in tqdm(study_ids):
    df = train[train['StudyInstanceUID'] == study_id].copy()
    nans = df['boxes'].isna()
    n_nans = nans.sum()
    if (n_nans > 0) and (n_nans < df.shape[0]):
        bad_images += list(df[nans]['id'])

100%|██████████| 232/232 [00:00<00:00, 525.59it/s]


In [25]:
len(bad_images)

217

In [26]:
bad_images[:5]

['04cc2f7f4c4b_image',
 '05c063f5cef5_image',
 '156cb1f5c689_image',
 '4c414b793562_image',
 'a5a364383f34_image']

In [23]:
study_id

'3795c5ddd545'

In [27]:
train_fixed = train[~train['id'].isin(bad_images)].copy()
train.shape, train_fixed.shape

((6334, 12), (6117, 12))

In [29]:
train_fixed.to_csv('/raid/siim-covid19-detection/train_image_level_folded_v3.csv', index=False)